<a href="https://colab.research.google.com/github/VedikaSingh1/DocAnalyzer/blob/master/RAG_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fastapi uvicorn python-multipart nest_asyncio pyngrok

In [ ]:
pip install langchain pymupdf faiss-cpu nltk langchain-community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
from fastapi import FastAPI, UploadFile, Form
from fastapi.responses import JSONResponse
import os
from google.colab import userdata
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import langchain
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from tempfile import NamedTemporaryFile

In [ ]:
app = FastAPI()

vectorstore = None
chain = None

In [ ]:
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')

In [ ]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-8b",
    temperature=0.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
@app.post("/upload_pdf/")
async def upload_pdf(file: UploadFile):
    """
    Endpoint to upload a PDF and initialize the FAISS vectorstore.
    """
    global vectorstore, chain

    # Save the uploaded file temporarily
    with NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(await file.read())
        temp_path = temp_file.name

    # Load and process the PDF
    loader = PyMuPDFLoader(temp_path)
    data = loader.load()

    # Split data into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=500
    )
    docs = text_splitter.split_documents(data)

    # Create vectorstore
    vectorstore = FAISS.from_documents(docs, gemini_embeddings)

    # Initialize retrieval chain
    chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

    # Clean up temporary file
    os.remove(temp_path)

    return JSONResponse({"message": "PDF uploaded and processed successfully!", "num_chunks": len(docs)})

In [ ]:
@app.post("/query/")
async def query_paper(question: str = Form(...)):
    """
    Single endpoint to handle a query and return the response.
    """
    global chain

    if chain is None:
        # Return an error if no PDF has been uploaded yet
        return JSONResponse(
            {"error": "No PDF has been uploaded yet. Please upload a PDF first."},
            status_code=400
        )

    try:
        # Run the query through the chain
        langchain.debug = True  # Enable debugging for detailed logs
        result = chain({"question": question}, return_only_outputs=True)

        # Return the result as JSON
        return JSONResponse({"result": result})
    except Exception as e:
        # Handle unexpected errors
        return JSONResponse(
            {"error": "An error occurred while processing the query.", "details": str(e)},
            status_code=500
        )

In [ ]:
ngrok.set_auth_token(userdata.get('NGROK_AUTH'))
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://647f-34-125-191-5.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [610]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     182.66.218.123:0 - "POST /upload_pdf/ HTTP/1.1" 200 OK


<ipython-input-17-3d316781827e>:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": question}, return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is backpropagation?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "4\n3) Backward Propagation and Weight Updates:\n• Compute gradients of the loss with respect to the weights\nby the usage of backpropagation:\n∂L\n∂w = Backprop(L, w)\n(18)\n• Update the weights using an optimizer (Adam):\nwt+1 = wt −η ∂L\n∂wt\n(19)\nwhere η stands for learning rate.\n4) Validation:\n• Evaluate the model against the validation data after each\nepoch.\n• Compute the accuracy and loss on the validation set:\nAccuracy = Number of Correct Predictions\nTotal Number of Predictions\n(20)\nVI. RESULTS",
      "question": "What is backpropagation?"
    },
    {
  

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [610]
